In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop = stopwords.words()

[nltk_data] Downloading package stopwords to
[nltk_data]     /raid/alebedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
X_train = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv")
y_train = X_train["label"]

In [7]:
X_train.head()

,ID,url,title,label
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",0
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,0
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,0
3,3,colorbox.spb.ru,Не Беси Меня Картинки,0
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,0


In [8]:
X_test.head()

,ID,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [10]:
del X_train["ID"]
del X_train["label"]

In [11]:
SEED = 42
pipeline = Pipeline(steps=[
    (
        'features', 
        ColumnTransformer([
            ('url', TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 4), min_df=3, max_df=0.75), 'url'),
            ('title', CountVectorizer(stop_words=stop), 'title')
        ])
    ),
    ('clf', LogisticRegression(random_state=SEED, solver='lbfgs', class_weight='balanced'))
])

pipeline.fit(X_train, y_train)

/raid/alebedev/myenvtr/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aldiz', 'baiknya', 'baizik', 'berkali', 'bukatzeko', 'edota', 'eze', 'ezpabere', 'ezpada', 'ezperen', 'gainera', 'gainerontzean', 'guztiz', 'hainbestez', 'horra', 'kali', 'kurangnya', 'mata', 'olah', 'onların', 'ordea', 'osterantzean', 'printr', 'sekurang', 'setidak', 'tama', 'tidaknya', 'δι', 'арбаң', 'арсалаң', 'афташ', 'бай', 'бале', 'баски', 'батыр', 'баҳри', 'болои', 'бүгжең', 'бұтыр', 'валекин', 'вақте', 'вой', 'вуҷуди', 'гар', 'гарчанде', 'далаң', 'даме', 'ербелең', 'жалт', 'жұлт', 'карда', 'кошки', 'куя', 'күңгір', 'кӣ', 'магар', 'майлаш', 'митың', 'модоме', 'нияти', 'онан', 'оре', 'паһ', 'рӯи', 'салаң', 'сар', 'сұлаң', 'сұрт', 'тарбаң', 'тразе', 'ту', 'тыржың', 'тұрс', 'хом', 'хуб', 'чаро', 'чи', 'чун', 'чунон', 'шарте', 'шұңқ', 'ыржың', 'қадар', 'қайқаң', 'қалт', 'қаңғы

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('url',
                                                  TfidfVectorizer(analyzer='char_wb',
                                                                  max_df=0.75,
                                                                  min_df=3,
                                                                  ngram_range=(2,
                                                                               4)),
                                                  'url'),
                                                 ('title',
                                                  CountVectorizer(stop_words=['إذ',
                                                                              'إذا',
                                                                              'إذما',
                                                                              'إذن',
                                                  

In [12]:
y_predict = pipeline.predict(X_test)

In [16]:
X_test["label"] = y_predict

In [19]:
X_test

,ID,url,title,label
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...,0
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ...",0
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12,0
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ...",1
4,135313,2gis.ru,67,0
...,...,...,...,...
165373,300682,etp.armtek.ru,Armtek - запчасти для грузовых и легковых авто...,0
165374,300683,mail.ru,"Лилия Якупова - Караганда, Карагандинская обла...",0
165375,300684,xn----8sbnqchpeeeth.xn--p1ai,Администрация Лесного района Тверской области ...,0
165376,300685,www-sunhome-ru.cdn.ampproject.org,Сонник Изменение сознания. К чему снится Измен...,0


In [ ]:
X_test[["id", "target"]].to_csv("LogisticRegression", index=False)
!cat ml_baseline.csv | head

In [ ]:
test_df["label"] = test_df["title"].apply(lambda x: f(str(x)))

In [20]:
X_test[["ID", "label"]].to_csv("LogisticRegression.csv", index=False)

!cat LogisticRegression.csv | head

ID,label
135309,0
135310,0
135311,0
135312,1
135313,0
135314,0
135315,0
135316,0
135317,0
cat: write error: Broken pipe
